In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.optimize as spop
from statsmodels.tsa.stattools import coint
from sklearn.model_selection import ParameterGrid

# Ensure plots display in Jupyter Notebook
%matplotlib inline

# Parameters
universe = ['JPM', 'C', 'GS', 'MS', 'BAC', 'WFC', 'USB', 'AXP', 'COF', 'PNC']  # Expanded universe of stocks
start = '2019-01-01'  # Extended start date
end = '2021-03-08'
fee = 0.001
window = 252
t_threshold = -2.5
coint_threshold = 0.1  # Increased p-value threshold for cointegration

# Fetch data
data = pd.DataFrame()
for stock in universe:
    prices = yf.download(stock, start, end)
    if 'Close' in prices.columns:
        data[stock] = prices['Close']
    else:
        print(f"Data for {stock} not available")

# Check if data is correctly downloaded
print(data.head())

# Function to identify cointegrated pairs
def find_cointegrated_pairs(data, coint_threshold):
    n = data.shape[1]
    score_matrix = np.zeros((n, n))
    pvalue_matrix = np.ones((n, n))
    keys = data.columns
    pairs = []
    
    for i in range(n):
        for j in range(i+1, n):
            S1 = data[keys[i]]
            S2 = data[keys[j]]
            result = coint(S1, S2)
            score = result[0]
            pvalue = result[1]
            score_matrix[i, j] = score
            pvalue_matrix[i, j] = pvalue
            if pvalue < coint_threshold:
                pairs.append((keys[i], keys[j]))
    
    return score_matrix, pvalue_matrix, pairs

# Find cointegrated pairs
scores, pvalues, pairs = find_cointegrated_pairs(data, coint_threshold)
print(f"Cointegrated pairs: {pairs}")

# Backtesting function
def backtest_pair(data, stock1, stock2, fee, window, t_threshold):
    returns = pd.DataFrame()
    returns[stock1] = np.append(data[stock1][1:].reset_index(drop=True) / data[stock1][:-1].reset_index(drop=True) - 1, 0)
    returns[stock2] = np.append(data[stock2][1:].reset_index(drop=True) / data[stock2][:-1].reset_index(drop=True) - 1, 0)
    
    gross_returns = np.array([])
    net_returns = np.array([])
    t_s = []
    
    for t in range(window, len(data)):
        def unit_root(b):
            a = np.average(data[stock2][t-window:t] - b * data[stock1][t-window:t])
            fair_value = a + b * data[stock1][t-window:t]
            diff = np.array(fair_value - data[stock2][t-window:t])
            diff_diff = diff[1:] - diff[:-1]
            reg = sm.OLS(diff_diff, diff[:-1])
            res = reg.fit()
            return res.params[0] / res.bse[0]
        
        try:
            res1 = spop.minimize(unit_root, data[stock2][t] / data[stock1][t], method='Nelder-Mead')
            t_opt = res1.fun
            b_opt = float(res1.x)
            a_opt = np.average(data[stock2][t-window:t] - b_opt * data[stock1][t-window:t])
            fair_value = a_opt + b_opt * data[stock1][t]
            
            if t == window:
                old_signal = 0
            
            if t_opt > t_threshold:
                signal = 0
                gross_return = 0
            else:
                signal = np.sign(fair_value - data[stock2][t])
                gross_return = signal * returns[stock2][t] - signal * returns[stock1][t]
            
            fees = fee * abs(signal - old_signal)
            net_return = gross_return - fees
            gross_returns = np.append(gross_returns, gross_return)
            net_returns = np.append(net_returns, net_return)
            t_s = np.append(t_s, t_opt)
            
            old_signal = signal
        except Exception as e:
            print(f"Error at t={t}: {e}")
    
    return gross_returns, net_returns, t_s

# Run backtests for all cointegrated pairs
results = {}
for pair in pairs:
    stock1, stock2 = pair
    gross_returns, net_returns, t_s = backtest_pair(data, stock1, stock2, fee, window, t_threshold)
    results[pair] = {
        'gross_returns': gross_returns,
        'net_returns': net_returns,
        't_stats': t_s
    }

# Evaluate performance
for pair, result in results.items():
    net_returns = result['net_returns']
    cumulative_net_return = np.prod(1 + net_returns) - 1
    print(f"Pair: {pair}, Cumulative Net Return: {cumulative_net_return:.2f}")

    # Plot results
    plt.plot(np.append(1, np.cumprod(1 + result['gross_returns'])), label=f'Gross Returns {pair}')
    plt.plot(np.append(1, np.cumprod(1 + result['net_returns'])), label=f'Net Returns {pair}')
    plt.legend()
    plt.title(f'Performance of {pair}')
    plt.show()